# TP1 - Problema 2

### Grupo 11

Nelson Almeida a95652
<br>
Nuno Costa a97610

#### Problema 2

Na criptografia pós-quântica os reticulados inteiros (“hard lattices”) e os problemas a eles associados são uma componente essencial. Um reticulado inteiro pode ser definido por uma matriz $\;\mathsf{L} \in \mathbb{Z}^{m\times n}\;$ (com $\;m > n\;$) de inteiros e por um inteiro primo $\;q\geq 3\;$.


O chamado problema do vetor curto  (SVP) consiste  no cálculo de um vetor de inteiros
                                                   $$\;e\in \{-1,0,1\}^m\;$$
não nulo que  verifique a seguinte relação matricial
                                         $$
                                         \forall\,i < n\,\centerdot\,\sum_{j< m}\,e_j\,\times\,\mathsf{L}_{j,i}\;\equiv\;0\mod q
                                         $$


a) Pretende-se resolver o SVP por programação inteira dentro das seguintes condições

1. Os valores  $\,m\,,\,n\,,\,q\,$  são escolhidos com $\,n > 30\,$, $\,|m| > 1 + |n|\;$ e $\,|q| > |m|\,$. 

2. Os elementos $\;\mathsf{L}_{j,i}\;$ são gerados aleatória e uniformemente no intervalo inteiro $\,\{-d \cdots d\}$ sendo $\;d\equiv (q-1)/2\;$.

b) Pretende-se determinar em, em primeiro lugar, se existe um vetor $\,e\,$ não nulo (pelo menos um dos $\,e_j\,$ é diferente de zero). Se existir $\,e\,$ pretende-se calcular o vetor que minimiza o número de componentes não nulas.

Notas:                

- Se $\;x \ge 0\;$, representa-se por $\,|x|\,$ o tamanho de $\,x\,$ em bits:  o menor $\,\ell\,$ tal que $\,x < 2^\ell$.
    
- Um inteiro $\;x\;$ verifica $\;x \equiv 0 \mod q\;$  sse $\;x\;$ é um múltiplo de $\,q\,$. 
$$x \equiv 0 \mod q \;\quad \text{sse}\quad\; \exists\,k\in \mathbb{Z}\,\centerdot\, x \,=\,q\times k$$
    
Por isso, escrito de forma matricial, as relações que  determinam o vetor $\;e\neq 0\;$ são
        
$$\left\{\begin{array}{rcl}\exists\,e\in \{-1,0,1\}^m\,\centerdot\,\exists\,k\in \mathbb{Z}^n &\centerdot & e\times \mathsf{L} \;=\; q\,k \\ \exists\,i < n &\centerdot & e_i \,\neq\, 0 \end{array}\right.$$


In [85]:
from ortools.sat.python import cp_model
import random

model = cp_model.CpModel()

# Inputs
n = 4
m = 16
q = 32
d = (q - 1) // 2

In [86]:
# Inicialização da matriz
L = {}
for j in range(m):
    for i in range(n):
        L[j,i] = random.randint(-d, d+1)
        
# Vetor de alocação
e = [model.NewIntVar(-1, 1, f"e[{j}]") for j in range(m) ]

Passamos agora à modelação das restrições e à sua introdução no *model*.

A restrição

1. Verficar que o vetor não é nulo.

In [98]:
# Verificar se existe não nulos
model.Add(sum(e[j] for j in range(m)) != 0)

#vetor auxiliar onde serão guardados os valores presentes no vetor e transformados com a função módulo
u = [model.NewIntVar(0, 1, f'U[{j}]') for j in range(m) ]
for j in range(m):
    model.AddAbsEquality(u[j], e[j])

#Visto o vetor u ter o módulo do vetor e, caso o somatório do mesmo originine um valor diferente de 
model.Add(sum(u)>0)

for i in range(n):
    K = model.NewIntVar(-20, 20, f'K[{i}]')
    model.Add(sum(e[j] * L[j, i] for j in range(m)) == q * K)

NotImplementedError: calling abs() on a linear expression is not supported, please use CpModel.AddAbsEquality

Minimizar o número de componentes não nulas do vetor e

In [97]:
#o vetor u tem apenas valores positivos, visto ser gerado através do módulo de cada elemento do vetor e,
#e por isso para minimizar o número de componentes não nulas
#teremos de minimizar o valor originado pela soma de todos os elementos de u
model.Minimize(sum(u))

In [94]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for j in range(m):
        print(solver.Value(e[j]), end=' ')
else:
    print("not found")

0 0 0 0 0 -1 0 0 0 0 1 0 -1 0 1 -1 